Source: https://github.com/mylee16/onemap-api

In [1]:
import pandas as pd
import numpy as np
import requests

hdb_string = './resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv'

C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Randy\anaconda3\envs\IBM Data Science Project\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
df = pd.read_csv(hdb_string)

In [3]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113959 entries, 0 to 113958
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                113959 non-null  object 
 1   town                 113959 non-null  object 
 2   flat_type            113959 non-null  object 
 3   block                113959 non-null  object 
 4   street_name          113959 non-null  object 
 5   storey_range         113959 non-null  object 
 6   floor_area_sqm       113959 non-null  float64
 7   flat_model           113959 non-null  object 
 8   lease_commence_date  113959 non-null  int64  
 9   remaining_lease      113959 non-null  object 
 10  resale_price         113959 non-null  float64
dtypes: float64(2), int64(1), object(8)
memory usage: 9.6+ MB


In [5]:
df['Address'] = df['block'] + " " + df['street_name']
# Prints only every 20 entries, This reduces total dataset of 113959 to a much manageable number
# API only accepts certain format so having them fed into the API as 'Block number' 'Street Name ' works best
addresslist = list(df['Address'])[::20]

In [6]:
addresslist[:10]

['406 ANG MO KIO AVE 10',
 '119 ANG MO KIO AVE 3',
 '601 ANG MO KIO AVE 5',
 '1 BEDOK STH AVE 1',
 '2 BEDOK STH AVE 1',
 '142 BEDOK RESERVOIR RD',
 '722 BEDOK RESERVOIR RD',
 '182 BISHAN ST 13',
 '226 BT BATOK CTRL',
 '106 BT BATOK CTRL']

In [7]:
def getcoordinates(address):
    # get request with query
    req = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    # eval parse the string as a python statement. The resulting output is put into resultsdict
    resultsdict = eval(req.text)
    
    # the output is a dictionary. Fetches coordinates from results 
    if len(resultsdict['results'])>0:
        return resultsdict['results'][0]['LATITUDE'], resultsdict['results'][0]['LONGITUDE']
    else:
        pass

In [ ]:
%%time

coordinateslist= []
count = 0
failed_count = 0
for address in addresslist:
    try:
        if len(getcoordinates(address))>0:
            count = count + 1
            print('Extracting',count,'out of',len(addresslist),'addresses')
            coordinateslist.append(getcoordinates(address))
    except:
        count = count + 1           
        failed_count = failed_count + 1
        print('Failed to extract',count,'out of',len(addresslist),'addresses')
        coordinateslist.append(None)
print('Total Number of Addresses With No Coordinates',failed_count)

Extracting 1 out of 5698 addresses
Extracting 2 out of 5698 addresses
Extracting 3 out of 5698 addresses
Extracting 4 out of 5698 addresses
Extracting 5 out of 5698 addresses
Extracting 6 out of 5698 addresses
Extracting 7 out of 5698 addresses
Extracting 8 out of 5698 addresses
Extracting 9 out of 5698 addresses
Extracting 10 out of 5698 addresses
Extracting 11 out of 5698 addresses
Extracting 12 out of 5698 addresses
Extracting 13 out of 5698 addresses
Extracting 14 out of 5698 addresses
Extracting 15 out of 5698 addresses
Extracting 16 out of 5698 addresses
Extracting 17 out of 5698 addresses
Extracting 18 out of 5698 addresses
Extracting 19 out of 5698 addresses
Extracting 20 out of 5698 addresses
Extracting 21 out of 5698 addresses
Extracting 22 out of 5698 addresses
Extracting 23 out of 5698 addresses
Extracting 24 out of 5698 addresses
Extracting 25 out of 5698 addresses
Extracting 26 out of 5698 addresses
Extracting 27 out of 5698 addresses
Extracting 28 out of 5698 addresses
E

In [ ]:
df_coordinates = pd.DataFrame(coordinateslist)
df_combined = df.join(df_coordinates)
df_combined  = df_combined .rename(columns={0:'Latitude', 1:'Longitude'})

In [ ]:
df_combined.to_csv('hdb-property-information with coordinates.csv',index=False)